In [1]:
import json
import numpy as np
import wowsims

spell_id_lists = {
        '0' : "Melee",
        '23881' : "Bloodthirst",
        '1680' : "Whirlwind",
        '47475' : "Slam",
        '47450' : "Heroic Strike",
        '47471' : "Execute",
        '12867': "Deep Wounds",
        '12292' : "Death Wish",
        '1719' : "Recklessness",
        '64382' : "Shattering Throw",
        '54758' : "Engi Gloves",
        '2457' : "Battle Stance",
        '2458' : "Berserker Stance",
        '2687' : 'Bloodrage',
        '2825' : 'Bloodlust',
        '47465': 'Rend',
        '7384': 'Overpower',
        '44949': 'Whirlwind OH',
}

# Load Fury Raid Sim Request JSON

In [13]:
f = open('data/fury-human-bis-p3.json')
settings = json.load(f)

# Set environment and settings

In [14]:
from fury import *

def reset():
    wowsims.new(json.dumps(settings).encode('utf-8'))


# Iterations are currently capped at 3000
iterations = settings['simOptions']['iterations']

duration = settings['encounter']['duration']
reset()

# Default sim agent
Default sim agent is the hardcoded agent from the sim

In [4]:
settings['simOptions']['interactive'] = False
damages = np.array([])

for i in range(iterations):
    reset()
    while not wowsims.step():
        pass
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)

print(f'Average DPS: {damages.mean() / duration}')

Average DPS: 11782.327334307121


In [7]:
settings['simOptions']['interactive'] = False

reset()
while not wowsims.step():
    pass
cast_metrics = wowsims.getSpellMetrics()
for spell_id, metrics in cast_metrics.items():
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")

Melee: [1, 1, 1, 2, 6, 65]
Death Wish: [2]
Deep Wounds: [113]
Whirlwind: [12]
Recklessness: [1]
Bloodthirst: [28]
Battle Stance: [6]
Berserker Stance: [6]
Bloodrage: [3]
Bloodlust: [1]
Whirlwind OH: [12]
Heroic Strike: [59]
Rend: [4]
Execute: [4]
Slam: [18]
Engi Gloves: [3]
Shattering Throw: [1]
Overpower: [2]


# PPO Agent

## Load Trained PPO Model

In [2]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.registry import register_env
from fury_sim_env import FurySimEnv
from gymnasium import make
from gymnasium.envs.registration import register

def env_creator(env_config):
    return FurySimEnv(...)

register_env("FurySimEnv", env_creator)

config = PPOConfig()\
            .rollouts(num_rollout_workers=1)

algorithm = config.build(env="FurySimEnv")
algorithm.restore("models\Fury\PPO\PPO_FurySimEnv_b7fbb_00005_5_entropy_coeff=0.0100,kl_coeff=0.3000,lambda=1,num_sgd_iter=10,sgd_minibatch_size=500_2023-05-01_20-49-18\checkpoint_000022")

register(id="FurySimEnv", entry_point="fury_sim_env:FurySimEnv")
env = make("FurySimEnv")

c:\ProgramData\Anaconda3\envs\civ6_rl_rllib\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-01 20:15:39,208	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(RolloutWorker pid=28692) 2023-05-01 20:15:45,140	WARNING env.py:155 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
2023-05-01 20:15:45,273	WARNING util.py:67 -- Install gputil for GPU system monitoring.
2023-05-01 20:15:45,277	WARNING checkpoints.py:109 -- No `rllib_checkpoint.json` file found in checkpoint directory models\Fury\PPO\PPO_FurySimEnv_991e2_00010_10_entropy_coeff=0,kl_coeff=0.5000,lambda=0.9500,num_sgd_iter=20,sgd_minibatch_size=500_2023-05-01_19-09-47\checkpoint_000015\.! Trying to extract checkpoint inf

## Run PPO Model

In [3]:
for i in range(100):
    observation, info = env.reset()
    terminated = False
    reward = 0
    batch = []
    dps_results = np.array([])
    metrics_batch = []
    debug_logs = []
    debug_logs_batch = []
    while not terminated:
        action = algorithm.compute_single_action(observation)
        observation, reward, terminated, truncated, info = env.step(action)
        batch.append(np.concatenate((observation, [action], [reward])))
        debug_logs.append(info['debug log'])
    debug_logs_batch.append(debug_logs)
    dps_results = np.append(dps_results, info['dps'])
    metrics_batch.append(info['spell metrics'])

In [4]:
median = np.argsort(dps_results)[len(dps_results)//2]
print(f'Average DPS: {dps_results[median]}')

cast_metrics = metrics_batch[median]
for spell_id, metrics in cast_metrics.items():
    # Only one target, so we can just take the first one
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")

Average DPS: 6138.323719235898
Melee: [1, 21, 24]
Deep Wounds: [68]
Whirlwind: [5]
Bloodthirst: [5]
Bloodrage: [1]
Bloodlust: [1]
Whirlwind OH: [5]
Heroic Strike: [4]
Execute: [9]
Slam: [52]
Engi Gloves: [1]
Shattering Throw: [1]


### Debug Log

In [5]:
actions_id = {
    0: "Bloodthirst", 
    1: "Whirlwind",
    2: "Slam",
    3: "HeroicStrike",
    4: "Execute",
    5: "DeathWish",
    6: "Recklessness",
    7: "ShatteringThrow",
    8: "Bloodrage",
    9: "EngiGlove",
    10: "Bloodlust",
}

In [6]:
import pandas as pd

df = pd.DataFrame(debug_logs_batch[median], columns=['Timestamp', 'Action', 'Successful Cast', 'Damage Done', 'Total Damage Done', 'Rage'])
df['Action'] = df['Action'].map(actions_id)
df['DPS'] = df['Total Damage Done'].div(df['Timestamp'])
df

,Timestamp,Action,Successful Cast,Damage Done,Total Damage Done,Rage,DPS
0,0.010000,Slam,True,5343.576830,5343.576830,56.805632,534357.683047
1,1.520000,HeroicStrike,True,10609.969590,15953.546421,72.978699,10495.754224
2,1.520000,Slam,True,0.000000,15953.546421,72.978699,10495.754224
3,3.030000,Slam,True,9254.488217,25208.034638,57.978699,8319.483379
4,4.918250,Bloodlust,True,8177.141382,33385.176020,30.978699,6788.019806
...,...,...,...,...,...,...,...
102,118.607401,Execute,True,12064.557741,733882.282311,63.993515,6187.491483
103,120.801024,Slam,True,8989.875967,742872.158279,100.000000,6149.551848
104,122.311024,Execute,True,12000.409684,754872.567962,55.000000,6171.745975
105,124.311024,Execute,True,8188.737090,763061.305052,47.431564,6138.323719


In [18]:
df.to_csv("debug_logs/debug_log_1.csv")

# Spell Debug

In [4]:
reset()
Spells.register()
Auras.register()
TargetAuras.register()

settings['simOptions']['interactive'] = True
damages = np.array([])
spell_metrics = []

for i in range(3000):
    reset()
    while not wowsims.step():
        if wowsims.needsInput():
            wowsims.trySpell(Spells.HeroicStrike)
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)
    spell_metrics.append(wowsims.getSpellMetrics())

median = np.argsort(damages)[len(damages)//2]
print(f'Average DPS: {damages[median] / duration}')

Average DPS: 5136.616341010212


In [5]:
settings['simOptions']['interactive'] = True

reset()
while not wowsims.step():
    pass
cast_metrics = spell_metrics[median]
for spell_id, metrics in cast_metrics.items():
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")

Melee: [1, 1, 56]
Deep Wounds: [63]
Heroic Strike: [56]
